In [1]:

from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path

#give the file path to your audio file
audio_file_path = './uploads/VID-20220918-WA0000/audio/VID-20220918-WA0000.wav'
wav_fpath = Path(audio_file_path)

wav = preprocess_wav(wav_fpath)
encoder = VoiceEncoder("cpu")
_, cont_embeds, wav_splits = encoder.embed_utterance(wav, return_partials=True, rate=16)
print(cont_embeds.shape)



ModuleNotFoundError: No module named 'resemblyzer'

In [7]:
from spectralcluster import SpectralClusterer

clusterer = SpectralClusterer(
    min_clusters=1,
    max_clusters=100,
    # p_percentile=0.90,
    # gaussian_blur_sigma=1
    )

labels = clusterer.predict(cont_embeds)


def create_labelling(labels,wav_splits):
    from resemblyzer import sampling_rate
    times = [((s.start + s.stop) / 2) / sampling_rate for s in wav_splits]
    labelling = []
    start_time = 0

    for i,time in enumerate(times):
        if i>0 and labels[i]!=labels[i-1]:
            temp = [str(labels[i-1]),start_time,time]
            labelling.append(tuple(temp))
            start_time = time
        if i==len(times)-1:
            temp = [str(labels[i]),start_time,time]
            labelling.append(tuple(temp))

    return labelling
  
labelling = create_labelling(labels,wav_splits)

In [6]:
print(labelling)

[('0', 0, 11.96), ('1', 11.96, 17.24), ('0', 17.24, 19.1), ('1', 19.1, 19.22), ('0', 19.22, 19.28), ('1', 19.28, 19.34), ('0', 19.34, 20.9), ('1', 20.9, 20.96), ('0', 20.96, 21.02), ('1', 21.02, 23.84), ('0', 23.84, 27.14), ('1', 27.14, 29.48), ('0', 29.48, 31.58), ('1', 31.58, 37.88), ('0', 37.88, 40.4), ('1', 40.4, 41.42), ('0', 41.42, 43.34), ('1', 43.34, 48.8), ('0', 48.8, 51.02), ('1', 51.02, 53.06), ('0', 53.06, 58.76), ('1', 58.76, 59.48), ('0', 59.48, 59.78), ('1', 59.78, 59.84), ('0', 59.84, 61.64), ('1', 61.64, 61.7), ('0', 61.7, 61.88), ('1', 61.88, 62.0), ('0', 62.0, 64.22), ('1', 64.22, 68.72), ('0', 68.72, 69.08), ('1', 69.08, 69.2), ('0', 69.2, 69.5), ('1', 69.5, 81.14), ('0', 81.14, 81.2), ('1', 81.2, 81.26), ('0', 81.26, 81.32), ('1', 81.32, 81.5), ('0', 81.5, 83.12), ('1', 83.12, 87.32), ('0', 87.32, 87.62), ('1', 87.62, 87.68), ('0', 87.68, 87.8), ('1', 87.8, 87.92), ('0', 87.92, 87.98), ('1', 87.98, 89.12), ('0', 89.12, 90.74), ('1', 90.74, 94.22), ('0', 94.22, 96.6